In [2]:
import os
import openai

from getpass import getpass
print('Enter OpenAI API key:')
openai.api_key = getpass()

os.environ['OPENAI_API_KEY']=openai.api_key
!head '{"ftune_training_data_gpt.jsonl"}'

Enter OpenAI API key:


 ········


{"messages": [{"role": "system", "content": "Generate a JSON string that describes a PokÃ©mon trade or giveaway, including sender's and receiver's PokÃ©mon details such as the name, IV, gender, ability, hidden ability, region, language, and form of each Pokemon."}, {"role": "user", "content": "Masudo Ditto Max IVs\\nSV\\nLooking to trade a English Ditto 5/6 best IVs with a master ball for Japanese or German Max IVs Ditto\n"}, {"role": "assistant", "content": "{ \"Sender's Pokemon\" :  [ { \"Pokemon\" : \"Ditto\" , \"IV\" : [\"pvp - 0 atk max\" , \"pvp - 0 spd max\" , \"max\"] , \"nature\" : [] , \"gender\" : [] , \"ability\" : [] , \"hidden ability\" : [] , \"language\" : [\"JPN\"] , \"form\" : [] , \"special\" : [] , \"pokeball\" :[\"master\"] , \"shiny\" : false } ] , \"Requested Pokemon\" : [ { \"Pokemon\" : \"Ditto\" , \"IV\" : [\"pvp - 0 atk max\" , \"pvp - 0 spd max\" , \"max\"] , \"nature\" : [] , \"gender\" : [] , \"ability\" : [] , \"hidden ability\" : [] , \"language\" : [\"J

In [4]:
client = openai.OpenAI()

client.files.create(
  file=open("ftune_training_data_gpt.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-mLgIqD0diwzxkVAJm8Wt7MMP', bytes=37680, created_at=1712642276, filename='ftune_training_data_gpt.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [5]:
client.fine_tuning.jobs.create(
  training_file="file-mLgIqD0diwzxkVAJm8Wt7MMP", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-8TaDZrmlSOlx9XmhvJGWIK06', created_at=1712642295, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-CZlFzbDxdf1L4vLbz7wi9rvN', result_files=[], status='validating_files', trained_tokens=None, training_file='file-mLgIqD0diwzxkVAJm8Wt7MMP', validation_file=None, user_provided_suffix=None, seed=45923206, integrations=[])

In [6]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=30)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-8TaDZrmlSOlx9XmhvJGWIK06")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-8TaDZrmlSOlx9XmhvJGWIK06", limit=10)


# Cancel a job
# client.fine_tuning.jobs.cancel("my_job_id")

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-7wOxYdsP2N9GotIEyFg3vKTu', created_at=1712642320, level='info', message='Fine-tuning job started', object='fine_tuning.job.event', data=None, type='message'), FineTuningJobEvent(id='ftevent-6upW0Cslycpt9nxlv0GCc8Oi', created_at=1712642318, level='info', message='Files validated, moving job to queued state', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-4gLGchCqS9WLsUMZYCQXUulm', created_at=1712642295, level='info', message='Validating training file: file-mLgIqD0diwzxkVAJm8Wt7MMP', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-0cVA1UmRJNj9xnFvZZKKCN3T', created_at=1712642295, level='info', message='Created fine-tuning job: ftjob-8TaDZrmlSOlx9XmhvJGWIK06', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [25]:
def generate_jsonstring(finetuned_model):
  client = openai.OpenAI()
  prompt = """LF Zacian for Zamazenta
SWSH

I’m looking for Zacian and willing to trade Zamazenta if anyone is willing to give there’s up. If not I’m open to a touch trade to help my Pokédex. I was hoping to get Zacian to update my Pokemon home Pokédex.
"""
  system_message = {"role" : "system", "content" : "Generate a JSON string that describes a Pokémon trade or giveaway, including sender's and receiver's Pokémon details such as the name, IV, gender, ability, hidden ability, region, language, and form of each Pokemon."}
  response = client.chat.completions.create(
      model=finetuned_model,
      messages = [
          system_message,
          {"role" : "user", "content" : prompt}
      ],
      temperature=0.7,
      max_tokens=500,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["###"]
      )
  return response.choices[0].message.content.strip()

# Replace with your model's name
finetuned_model = "ft:gpt-3.5-turbo-0125:personal::9BurV9Wd"
print(generate_jsonstring(finetuned_model))

{ "Sender's Pokemon" :  [ { "Pokemon" : "Zamazenta" , "IV" : [] , "nature" : [] , "gender" : [] , "ability" : [] , "hidden ability" : [] , "language" : [] , "form" : [] , "special" : [] , "pokeball" :[] , "shiny" : false } ] , "Requested Pokemon" : [ { "Pokemon" : "Zacian" , "IV" : [] , "nature" : [] , "gender" : [] , "ability" : [] , "hidden ability" : [] , "language" : [] , "form" : [] , "special" : [] , "pokeball" : [] , "shiny" : false } ] , "version" : "SWSH" , "action" : "trade" }
